In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
import yfinance as yf
from fredapi import Fred
from datetime import date

In [3]:
today = pd.Timestamp(date.today())

In [4]:
df = pd.read_excel('sp500.xlsx', parse_dates=[0], index_col=0)

In [5]:
df

,Open,Close,Prediction
Date,,,
1992-02-03,408.790009,409.529999,-0.006658
1992-02-04,409.600006,413.850006,-0.005357
1992-02-05,413.880005,413.839996,-0.006448
1992-02-06,413.869995,413.820007,-0.006538
1992-02-07,413.820007,411.089996,-0.008424
...,...,...,...
2024-06-25,5460.729980,5469.299805,0.033399
2024-06-26,5460.709961,5477.899902,0.032249
2024-06-27,5473.589844,5482.870117,0.032131


In [6]:
# Create a column for daily returns
# Initialize variables to track the position
holding = 0  # Can be positive number of shares -'long' or negative number of shares - 'short'
entry_price = df['Open'][0]
slippage = 0.01
risk = 0.95   # the part of assets under the risk
cash = 1000
df['Holding'] = 0
df['Cash'] = cash
df['Assets'] = cash
df['Action'] = None

In [7]:
for i in range(len(df) - 1):
    action = []
    if df.iloc[i]['Prediction'] >= 0.0 and holding <= 0.0: 
        # We have to exit the short position and then buy long
#        print('short->long on the day', df.index[i])
        exit_price = df.iloc[i + 1]['Open'] * (1 + slippage)
        cash += exit_price * holding
        action = f'Buy {holding:.4f} at {exit_price:.4f}'
        holding = 0
            
        if cash < 0:
            print("You are bankrot on the day ", df.index[i] )
            break;
                
        # Buy long
        entry_price = df.iloc[i + 1]['Open'] * (1 + slippage)
        holding = risk * cash / entry_price
        cash -= risk * cash
        action += f' and {-holding:.4f} at {exit_price:.4f}'
        
        df.at[df.index[i + 1], 'Action'] = action
        
    elif df.iloc[i]['Prediction'] < 0.0 and holding >= 0.0:               
        # We have to sell the long position and then sell short
#        print('long->short on the day', df.index[i], 'holding', holding)
        
        exit_price = df.iloc[i + 1]['Open'] * (1 - slippage)
        cash += exit_price * holding
        action = f'Sell {holding:.4f} at {exit_price:.4f}'
        holding = 0
        
        # Sell short
        if cash < 0:
            print("You are bankrot on the day ", df.index[i] )
            break;
                
        # Enter short position
        entry_price = df.iloc[i + 1]['Open'] * (1 - slippage)
        holding = - risk * cash / entry_price
        cash += risk * cash
        action += f' and {holding:.4f} at {entry_price:.4f}'

        df.at[df.index[i + 1], 'Action'] = action

#    print('exit_price', exit_price, 'cash=', cash, 'entry_price', entry_price, 'holding',holding)

    df.at[df.index[i + 1], 'Holding'] = holding
    df.at[df.index[i + 1], 'Cash'] = cash
    df.at[df.index[i + 1], 'Assets'] = cash + holding * df.iloc[i]['Close']
    
                
# Save the DataFrame with the calculated returns to a CSV file
#df.to_csv('/mnt/data/trading_strategy_evaluation.csv')


In [8]:
df.to_excel('trades.xlsx')

In [9]:
roi = np.exp((np.log(df['Assets'][-1] / df['Assets'][0])) / ((df.index[-1] - df.index[0]).days/365) )

In [10]:
roi

1.2823767842977938

# The average return from simulated trades is 28% per year